In [8]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [5]:
data =load_breast_cancer()
feature_name = data.feature_names
X,y = data.data,data.target

ValueError: Shape of passed values is (569, 30), indices imply (30, 30)

In [6]:
n_tree = 150

In [22]:
X_train,X_test,Y_train,Y_test = train_test_split(X,y,random_state=0,test_size=0.3)


array([[1.149e+01, 1.459e+01, 7.399e+01, ..., 7.431e-02, 2.941e-01,
        9.180e-02],
       [1.049e+01, 1.861e+01, 6.686e+01, ..., 6.528e-02, 2.213e-01,
        7.842e-02],
       [1.225e+01, 1.794e+01, 7.827e+01, ..., 8.211e-02, 3.113e-01,
        8.132e-02],
       ...,
       [9.436e+00, 1.832e+01, 5.982e+01, ..., 5.052e-02, 2.454e-01,
        8.136e-02],
       [9.720e+00, 1.822e+01, 6.073e+01, ..., 0.000e+00, 1.909e-01,
        6.559e-02],
       [1.151e+01, 2.393e+01, 7.452e+01, ..., 9.653e-02, 2.112e-01,
        8.732e-02]])

In [ ]:
#决策树
from sklearn.tree import DecisionTreeClassifier
tree= DecisionTreeClassifier(random_state=0)
tree.fit(X_train,Y_train)
score = tree.score(X_test,Y_test)
score
#0.9122807017543859

In [ ]:
from sklearn.ensemble import RandomForestClassifier
trees = RandomForestClassifier(n_estimators=n_tree,random_state=0)
trees.fit(X_train,Y_train)
score = trees.score(X_test,Y_test)
score
#0.9590643274853801

In [ ]:
#OOB


#意义？
ensemble_clfs =[
    ("criterion = 'gini'",RandomForestClassifier(oob_score=True,criterion='gini')),
    ("criterion = 'entropy'",RandomForestClassifier(oob_score=True,criterion='entropy'))
]

In [ ]:
min_n_tree = 15
max_n_tree = 150
oob_error_dict = dict()
score_dict =dict()
for label,clf_rf in ensemble_clfs:
    oob_error_list = []
    score_list = []
    for i in range(min_n_tree,max_n_tree+1,5):
        clf_rf.set_params(random_state=0)
        clf_rf.set_params(n_estimators=i)
        clf_rf.fit(X_train,Y_train)
        oob_error_list.append(1-clf_rf.oob_score_)
        score_list.append(clf_rf.score(X_test,Y_test))
    oob_error_dict[label] = oob_error_list
    score_dict[label] = score_list
oob_error_dict

In [ ]:
import matplotlib.pyplot as plt
for label,error_list in oob_error_dict.items():
    x = range(min_n_tree,max_n_tree+1,5)
    y = error_list
    plt.plot(x,y,label =label)
    plt.legend(loc ='upper right')
plt.xlabel('n_estimators')
plt.ylabel('oob_error')
plt.show()

#unpack dict记得加items

In [ ]:
for label,score_list in score_dict.items():
    x = range(min_n_tree,max_n_tree+1,5)
    y = score_list
    plt.plot(x,y,label =label)
    plt.legend(loc ='upper right')
plt.xlabel('n_estimators')
plt.ylabel('score')
plt.show()

In [ ]:
#随机森林的参数研究：
from sklearn.ensemble import RandomForestClassifier
trees = RandomForestClassifier(n_estimators=60,criterion='entropy',oob_score=True,random_state=0)
trees.fit(X_train,Y_train)
print(trees.oob_score_)
print(trees.score(X_test, Y_test))

In [ ]:
#画出条形图：
n_feature = X.shape[1]
plt.barh(range(n_feature),trees.feature_importances_,align='center')
plt.yticks(range(n_feature),data.feature_names)
plt.xlabel('feature_importances_')
plt.ylabel('feature')




In [ ]:
tree_importance_sort_id = np.argsort(trees.feature_importances_)

n_feature = X.shape[1]
plt.barh(range(n_feature),trees.feature_importances_[tree_importance_sort_id],align='center')
plt.yticks(range(n_feature),data.feature_names[tree_importance_sort_id])
plt.xlabel('feature_importances_')
plt.ylabel('feature')


In [ ]:
data.feature_names[tree_importance_sort_id]

In [ ]:
#rf vs adaboost vs GBDT
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier
clf_rf = RandomForestClassifier(n_estimators=150,random_state=0)
clf_ada = AdaBoostClassifier(n_estimators=150,random_state=0)
clf_GBDT = GradientBoostingClassifier(n_estimators=150,random_state=0)


In [ ]:
from mlxtend.evaluate import bias_variance_decomp
mse,bias,var = bias_variance_decomp(clf_rf,X_train,Y_train,X_test,Y_test,loss='mse',random_seed=0)
mse_G,bias_G,var_G = bias_variance_decomp(clf_GBDT,X_train,Y_train,X_test,Y_test,loss='mse',random_seed=0)
mse_A,bias_A,var_A = bias_variance_decomp(clf_ada,X_train,Y_train,X_test,Y_test,loss='mse',random_seed=0)

In [ ]:
mse